# Getting the topic distribution for each article

- Start with smaller success_urls df with goal13 completions
- Run LDA on it
- Get topic names printed
- Get topic distributions per html element

## 1. importing test data

In [ ]:
import pickle
import pandas as pd


In [ ]:
with open('../04_Data/success_g13_corpus.pkl', 'rb') as file:
    success_corpus = pickle.load(file)

In [ ]:
with open('../04_Data/success_g13_id2word.pkl', 'rb') as file:
    success_id2word = pickle.load(file)

In [ ]:
with open('../04_Data/success_g13_train_bigram.pkl', 'rb') as file:
    success_bigram = pickle.load(file)

## 2. running the model 

In [ ]:
import gensim


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=success_corpus,
                                           id2word=success_id2word,
                                           num_topics=10,
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [31]:
lda_model.print_topics()

[(0,
  '0.018*"find" + 0.015*"datum" + 0.014*"also" + 0.013*"begin" + 0.012*"look" + 0.011*"one" + 0.010*"solut" + 0.010*"everi" + 0.010*"new" + 0.009*"creatur"'),
 (1,
  '0.034*"climat" + 0.023*"sloth" + 0.021*"hand" + 0.021*"individu" + 0.019*"bank" + 0.017*"would" + 0.017*"habitat" + 0.016*"group" + 0.015*"level" + 0.014*"want"'),
 (2,
  '0.032*"emiss" + 0.029*"energi" + 0.016*"lca" + 0.014*"process" + 0.013*"life_cycl" + 0.012*"long" + 0.012*"product" + 0.011*"bitcoin" + 0.011*"blockchain" + 0.010*"scope"'),
 (3,
  '0.027*"approach" + 0.022*"result" + 0.020*"act" + 0.015*"economi" + 0.014*"carbon_footprint" + 0.013*"drive" + 0.013*"transit" + 0.012*"effici" + 0.012*"associ" + 0.012*"stakehold"'),
 (4,
  '0.030*"credit" + 0.028*"entir" + 0.023*"month" + 0.021*"away" + 0.014*"read" + 0.013*"oper" + 0.013*"repres" + 0.012*"live" + 0.012*"happen" + 0.011*"still"'),
 (5,
  '0.034*"plastic" + 0.032*"posit" + 0.023*"man" + 0.021*"water" + 0.019*"planetwid" + 0.017*"increas" + 0.016*"woman

## 3. Topic titles and distributions

In [ ]:
with open('../04_Data/success_g13.pkl', 'rb') as file:
    success_g13 = pickle.load(file)
    
success_g13.head(10)

In [ ]:
# displays the 10 topics and their top 10 words and weights

model_topics = lda_model.show_topics(formatted=False)
model_topics


In [ ]:
topic_titles = []

for i in model_topics:
    index, word_list = i
    word, prob = word_list[0]
    word1, prob1 = word_list[1]
    topic_title = f'{word}_{word1}'
    topic_titles.append(topic_title)
    

In [ ]:
topic_titles

In [ ]:
topic_dist = []

for i in range(len(success_g13)):
    top_topics = lda_model.get_document_topics(success_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(10)]
    topic_vec.extend([success_g13.index[i]]) 
    topic_dist.append(topic_vec)
    

In [ ]:
topic_dist

## 4. One-hot encoding topics to df

In [ ]:
# this can probably be part of the for loop that creates topic_dist to avoid this code repetition

top_0 = [topic[0] for topic in topic_dist]
top_1 = [topic[1] for topic in topic_dist]
top_2 = [topic[2] for topic in topic_dist]
top_3 = [topic[3] for topic in topic_dist]
top_4 = [topic[4] for topic in topic_dist]
top_5 = [topic[5] for topic in topic_dist]
top_6 = [topic[6] for topic in topic_dist]
top_7 = [topic[7] for topic in topic_dist]
top_8 = [topic[8] for topic in topic_dist]
top_9 = [topic[9] for topic in topic_dist]


In [ ]:
topic_titles

In [ ]:
success_g13 = success_g13.assign(find_datum=top_0, climat_sloth=top_1, emiss_energi=top_2, 
                                   approach_result=top_3, credit_entir=top_4, plastic_posit=top_5, 
                                   bear_polar=top_6, go_wast=top_7, alreadi_earli=top_8, 
                                   sustain_climat_chang=top_9)


In [ ]:
success_g13.head()


## 5. LDA model function
Df goes in, df with topic distribution comes out

### 5.1 setting up the LDA model

In [ ]:
def run_lda(lda_corpus, lda_id2word):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=lda_corpus,
                                                id2word=lda_id2word,
                                                num_topics=10,
                                                random_state=42,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
    

    return lda_model

### 5.2 Getting the topic titles

In [ ]:
def topic_titles(lda_model):
    
    model_topics = lda_model.show_topics(formatted=False)
    
    topic_titles = []
    
    for i in model_topics:
        index, word_list = i
        word, prob = word_list[0]
        word1, prob1 = word_list[1]
        topic_title = f'{word}_{word1}'
        topic_titles.append(topic_title)
    
    return topic_titles

### 5.3 Get topic distribution

In [ ]:
def topic_distribution(df, lda_model, corpus):
    
    topic_dist = []
    
    for i in range(len(df)):
        top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(10)]
        topic_vec.extend([df.index[i]]) 
        topic_dist.append(topic_vec)
        
    return topic_dist

### 5.4 Add topic distributions to df

In [ ]:
def get_lda_results(topic_titles, topic_dist, df):
    
    for i, value in enumerate(topic_titles):
        topic_col = [topic[i] for topic in topic_dist]
        df[value] = topic_col
    
    return df


### 5.5 Final function

In [ ]:
def lda_results_to_df(corpus, id2word, df):
    
    lda_model = run_lda(corpus, id2word)
    titles = topic_titles(lda_model)
    top_dist = topic_distribution(df, lda_model, corpus)
    df = get_lda_results(titles,top_dist, df)
    
    return df
    

## 6. Using function on big full academy df

In [ ]:
with open('../04_Data/full_academy_corpus.pkl', 'rb') as file:
    academy_corpus = pickle.load(file)

with open('../04_Data/full_academy_id2word.pkl', 'rb') as file:
    academy_id2word = pickle.load(file)

with open('../04_Data/academy_posts.pkl', 'rb') as file:
    academy_df = pickle.load(file)

In [ ]:
full_academy = lda_results_to_df(academy_corpus, academy_id2word, academy_df)
full_academy.head(10)

In [ ]:
# with open('../04_Data/lda_separated_posts.pkl', 'wb') as fa:
#    pickle.dump(full_academy, fa, protocol=pickle.HIGHEST_PROTOCOL)

## LDA interactive visualisation

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [32]:
# Visualize the topics

pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, success_corpus, success_id2word)
pyLDAvis.display(vis)

In [ ]:
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=academy_corpus,
                                           id2word=academy_id2word,
                                           num_topics=10,
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
pyLDAvis.enable_notebook(sort=True)
vis2 = pyLDAvis.gensim.prepare(lda_model2, academy_corpus, academy_id2word)
pyLDAvis.display(vis2)